## Asteroid conjunction finder

Find close approaches between asteroid positions (obtained dynamically from MPC ephemeris service) and NGC DSO catalog (cached locally from Vizier table)

In [53]:
# Parameters, edit to taste

# Date & time for ephemeris lookup

OBS_TIME = '2025-01-13 20:00:00'

# mimimum angular size of DSO to be visible in image
DSO_MIN_SIZE = 2  # 5% of Seestar S50 FoV width

# maximum magnitude for DSO to be visible 
DSO_MAX_MAG = 15  # approx limiting magnitude for Seestar S50

# Max angular distance between asteroid and DSO
MAX_SEPARATION_ARCMIN = 40 # approx frame width in Seestar S50

In [63]:
# list of largest asteroids to search

ASTEROIDS = ['Ceres',
                      '4 Vesta',
                       '2 Pallas',
                       '10 Hygiea',
                       '704 Interamnia',
                       '52 Europa',
                       '511 Davida',
                       '87 Sylvia',
                       '15 Eunomia',
                       '31 Euphrosyne',
                       '65 Cybele',
                       '624 Hektor',
                       '3 Juno',
                       '451 Patientia',
                       '107 Camilla',
                       '324 Bamberga',
                       '16 Psyche',
                       '88 Thisbe',
                       '48 Doris',
                       '19 Fortuna',
                       '121 Hermione',
                       '24 Themis',
                       '94 Aurora',
                       '29 Amphitrite',
                       '13 Egeria',
                       '130 Elektra',
                       '7 Iris',
                       '6 Hebe',
                       '375 Ursula',
                       '702 Alauda',
                       '45 Eugenia',
                       '41 Daphne',
                       '423 Diotima',
                       '259 Aletheia',
                       '372 Palma',
                       '9 Metis',
                       '532 Herculina',
                       '354 Eleonora',
                       '128 Nemesis'
]

In [ ]:
# Fetch ngc catalog from CDS vizier server
# This code only needs to be run once to cache the table locally

# from astroquery.vizier import Vizier

# vizier = Vizier()
# vizier.ROW_LIMIT = -1

# ngc_tables = vizier.get_catalogs('VII/118/ngc2000')

# # save ngc table locally for future reference

# ngc_tables[0].write('./ngc_catalog.ecsv')


In [ ]:
# create SkyCoord catalog of ngc DSOs for searching

from astropy.io import ascii

# load locally cached ngc table 
ngc_table = ascii.read('./ngc_catalog.ecsv')

# filter table to size and magnitude limits

filtered_ngc_table = ngc_table[ngc_table['size'] > DSO_MIN_SIZE]
filtered_ngc_table = filtered_ngc_table[filtered_ngc_table['mag'] < DSO_MAX_MAG]

In [38]:
from astropy.coordinates import SkyCoord
import astropy.units as u

ngc_catalog = SkyCoord(ra = filtered_ngc_table['RAB2000'], dec = filtered_ngc_table['DEB2000'],
                       unit = (u.hourangle, u.deg))



In [ ]:
# Query MPC Horizons service for asteroid ephemeris data

from astroquery.mpc import MPC
from astropy.table import vstack

ephemerides = [MPC.get_ephemeris(a, start = OBS_TIME, number = 1) for a in ASTEROIDS]
ephemerides = vstack(ephemerides)

# create SkyCoord catalog for search
ephemerides_cat = SkyCoord(ra = ephemerides['RA'], dec = ephemerides['Dec'], unit = u.deg)

In [ ]:
# Find sky conjunctions of asteroid positions and NGC catalog

idx_ngc, idx_asteroid, sep2d, _ = ephemerides_cat.search_around_sky(ngc_catalog, seplimit = MAX_SEPARATION_ARCMIN * u.arcmin)

for ingc, ia, sep in zip(idx_ngc, idx_asteroid, sep2d):
    print('NGC',filtered_ngc_table[ingc]['Name'],
          filtered_ngc_table[ingc]['Type'],
          '<>',
          ASTEROIDS[ia],
          '=',
        sep)

NGC 428 Gx <> 107 Camilla = 0d29m28.0442s
NGC 514 Gx <> 10 Hygiea = 0d22m10.1012s
NGC 926 Gx <> 511 Davida = 0d48m55.9664s
NGC 936 Gx <> 511 Davida = 0d38m12.1036s
NGC 941 Gx <> 511 Davida = 0d32m00.7319s
NGC 955 Gx <> 511 Davida = 0d36m11.1529s
NGC I 443 Nb <> 88 Thisbe = 0d21m07.6382s
NGC 5705 Gx <> 451 Patientia = 0d25m35.5491s
NGC 5713 Gx <> 451 Patientia = 0d49m29.9903s
NGC 5719 Gx <> 451 Patientia = 0d47m38.8117s
NGC 7309 Gx <> 16 Psyche = 0d54m47.2041s
